In [135]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.12f}'.format})

<Figure size 640x480 with 0 Axes>

In [136]:
fF = 1e-15
nH = 1e-9

In [342]:

def resonator_C_resonator(Cc):
    # if Cc == 0:
    #     C = sq.Capacitor(1, 'fF')
    #     L = sq.Inductor(1,  'nH')
    #     elements = {(0, 1): [L, C],
    #                 (0, 2): [L, C], }
    #     return sq.Circuit(elements)
    # 
    # else:
    Cc = sq.Capacitor(Cc, 'fF')
    C = sq.Capacitor(1,   'fF')
    L = sq.Inductor(1,    'nH')
    elements = {(0, 1): [L, C],
                (0, 2): [L, C],
                (1, 2): [Cc], }
    return sq.Circuit(elements)


def resonator(Cc):
    if Cc == 0:
        C = sq.Capacitor(1, 'fF')
        L = sq.Inductor(1,  'nH')
        elements = {(0, 1): [L, C], }
        return sq.Circuit(elements)

    else:
        C = sq.Capacitor(1/(1+1/Cc), 'fF')
        L = sq.Inductor (1, 'nH')
        elements = {(0, 1): [L, C], }
        return sq.Circuit(elements)
    # 
    # Cc = sq.Capacitor(Cc, 'fF')
    # C = sq.Capacitor(1, 'fF')
    # L = sq.Inductor(1,  'nH')
    # elements = {(0, 1): [L, C, Cc], }
    return sq.Circuit(elements)
    

In [343]:
n_eig = 6
trunc_num = 40
Cc = 1

In [344]:
coupled_res = resonator_C_resonator(Cc=Cc)
res = resonator(Cc=Cc)

uncoupled_res = resonator_C_resonator(Cc=0)
res_0 = resonator(Cc=0)

coupled_res.set_trunc_nums([trunc_num, trunc_num])
uncoupled_res.set_trunc_nums([trunc_num, trunc_num])
res.set_trunc_nums([trunc_num])
res_0.set_trunc_nums([trunc_num])

_ = coupled_res.diag(n_eig)
_ = uncoupled_res.diag(n_eig)
_ = res.diag(n_eig)
_ = res_0.diag(n_eig)

In [345]:
coupled_res.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [346]:
uncoupled_res.description()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [347]:
Q_0 = res_0._charge_op_isolated(0)
Φ_0 = res_0._flux_op_isolated(0)

Q = res._charge_op_isolated(0)
Φ = res._flux_op_isolated(0)

In [348]:
H_res_0 = 1 / 2 * (1/fF)               * Q_0 ** 2  + 1 / 2 * (1/nH) * Φ_0 ** 2
H_res   = 1 / 2 * (1/fF) * (1 + 1/Cc)  * Q ** 2    + 1 / 2 * (1/nH) * Φ   ** 2# 
# H_res_0 = res_0.hamiltonian()
# H_res   = res.hamiltonian()

I = qt.identity(H_res.shape[0])

H_unc_res = qt.tensor(H_res_0, I) + qt.tensor(I, H_res_0)

H_coup_res = qt.tensor(H_res, I) + qt.tensor(I, H_res) - (1 / (Cc*fF)) * qt.tensor(Q, Q)

In [349]:
E_0 = sq_ext.diag(H_res, n_eig=n_eig, out='GHz')[0]
E_0 - E_0[0]

array([0.000000000000, 225.079079039277, 450.158158078555, 675.237237117834, 900.316316157112, 1125.395395196392])

In [350]:
res.diag(n_eig)[0]

array([-0.000000000001, 225.079079039051, 450.158158078104, 675.237237117157, 900.316316156212, 1125.395395195260])

In [336]:
E_0 = sq_ext.diag(H_unc_res,n_eig=n_eig, out='GHz')[0]
E_0 - E_0[0]

array([0.000000000000, 159.154943091820, 159.154943091820, 318.309886183633, 318.309886183638, 318.309886183639])

In [333]:
coupled_res.efreqs

array([0.000000000001, 91.888149236954, 159.154943091817, 183.776298473902, 251.043092328770, 275.664447710857])

In [334]:
uncoupled_res.efreqs

array([0.000000000000, 159.154943091821, 159.154943091822, 318.309886183632, 318.309886183640, 318.309886183641])

In [335]:
(E_0 - E_0[0]) / coupled_res.efreqs


array([0.000000000000, 0.000000053740, 5.074360504248, 4.394525167449, 3.271387114762, 2.979198602954])

In [198]:
(E_0 - E_0[0]) / np.sqrt(3)


array([0.000000000000, 91.888149236969, 159.154943091898, 183.776298473935, 251.043092328866, 275.664447710905])